## Q2. 
Work on the red wine dataset: Wine.db
The data of red wine contains four tables: Wine, Winery, Grape, and Rater.

In [5]:
import os
import sys
import pandas as pd
import sqlite3
import my_packages.Sql_table_details
import xlsxwriter

# Paths and directory
current_directory = os.getcwd()
wine = os.path.join(current_directory, "01-data/01_wine_Project", "wine.db")


# function that checks table details
my_packages.Sql_table_details.display_table_details(wine)
#my_packages.Sql_table_details.sample_table(wine,'winery',10)


Table: Grape
+-----+---------+---------+---------+------------+----+
| cid |   name  |   type  | notnull | dflt_value | pk |
+-----+---------+---------+---------+------------+----+
|  0  |    id   | INTEGER |    0    |    None    | 1  |
|  1  | variety |   TEXT  |    0    |    None    | 0  |
+-----+---------+---------+---------+------------+----+

Table: Winery
+-----+----------+---------+---------+------------+----+
| cid |   name   |   type  | notnull | dflt_value | pk |
+-----+----------+---------+---------+------------+----+
|  0  |    id    | INTEGER |    0    |    None    | 1  |
|  1  | country  |   TEXT  |    0    |    None    | 0  |
|  2  | province |   TEXT  |    0    |    None    | 0  |
|  3  |  winery  |   TEXT  |    0    |    None    | 0  |
+-----+----------+---------+---------+------------+----+

Table: Rater
+-----+----------------+---------+---------+------------+----+
| cid |      name      |   type  | notnull | dflt_value | pk |
+-----+----------------+---------+------

## 2A. 
Grapes are the raw material for making red wine. Which country has the most varieties
of grapes, please report the name of the country and the number of types of grapes

In [6]:
GrapeQuery = """
select 
w2.country,
count(distinct g.variety) as[count of variety]
from Wine w1
 join Winery w2 on w1.wid=w2.id
 join Grape g on  w1.gid=g.id

group by
w2.country

order by count(distinct g.variety) desc
"""
Grapes=my_packages.Sql_table_details.run_query_from_db(wine,GrapeQuery)
#print(Grapes)


+------------------------+------------------+
|        country         | count of variety |
+------------------------+------------------+
|           US           |       294        |
|         Italy          |       174        |
|         France         |       160        |
|         Spain          |       139        |
|        Portugal        |       113        |
|       Australia        |        91        |
|         Chile          |        68        |
|       Argentina        |        68        |
|        Austria         |        56        |
|         Israel         |        54        |
|      South Africa      |        53        |
|         Greece         |        45        |
|      New Zealand       |        34        |
|         Canada         |        33        |
|        Germany         |        32        |
|        Slovenia        |        29        |
|        Hungary         |        29        |
|        Romania         |        27        |
|        Bulgaria        |        

## 2B. 
According to the guide published by wine enthusiasts, red wine can be divided into four
classes based on its price: everyday wine, mid-range wine, fine wine, and super fine wine. If the
price of red wine is higher than or equal to $340, it will be classified as super fine wine. Please
report the average price and points of the super fine wines.

In [7]:
finewineQuery = """
select
avg(points), 
avg(price)
From Wine w
where price >=340
order by price desc

"""
FineWine=my_packages.Sql_table_details.run_query_from_db(wine,finewineQuery)
#print(FineWine)

+-------------------+-------------------+
|    avg(points)    |     avg(price)    |
+-------------------+-------------------+
| 95.39755351681957 | 629.1896024464831 |
+-------------------+-------------------+


## 2C.
C. A rater evaluates the score and price of the wine fairly and unbiasedly based on the
quality of the wine. Please write SQL code and python code to analyse the data statistics of points
and prices of wine evaluated by each taster and use the format rater_name.xlsx to export the data
statistics.

In [47]:
import pandas as pd

# SQL query
RaterQuery = """
SELECT
    r.name AS [Rater],
    w.points , 
    w.price
FROM Wine w
JOIN Rater r ON w.rid = r.id
"""

# Run SQL query and store the result in Raters_table
Raters_table = my_packages.Sql_table_details.run_query_from_db(wine, RaterQuery, print_table=False)

# Function to calculate statistics
def calculate_statistics(data, column_name):
    grouped_statistics = data.groupby('Rater')[column_name].describe()
    median_series = data.groupby('Rater')[column_name].median()

    grouped_statistics = grouped_statistics.reset_index()
    grouped_statistics['median'] = median_series.values

    grouped_statistics = grouped_statistics[[
        'Rater', 'count', 'mean', 'median', 'std', 'min', '25%', '50%', '75%', 'max']]

    return grouped_statistics

# Columns to process
columns_to_process = ['price','points']

# Initialize an empty DataFrame to store combined statistics
combined_statistics = None

# Loop through the columns and calculate statistics for each
for idx, column in enumerate(columns_to_process):
    statistics = calculate_statistics(Raters_table, column)
    
    if idx == 1:
        # Drop the 'Rater' column after the first output
        statistics = statistics.drop(columns=['Rater'])
    
    statistics.rename(columns={col: f'{column} {col}' for col in statistics.columns}, inplace=True)
    
    if combined_statistics is None:
        combined_statistics = statistics
    else:
        # Merge based on the index (no 'on' parameter) with a blank column separator
        combined_statistics = pd.concat([combined_statistics, pd.DataFrame(columns=['']), statistics], axis=1)

# Rename the first column from the 'columns_to_process' list to "Rater" after the final append
if combined_statistics is not None:
    combined_statistics.rename(columns={combined_statistics.columns.values[0]: 'Rater'}, inplace=True)

# Create an Excel writer object
output_file = current_directory + '/02-output/Raters_names_combined.xlsx'
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    combined_statistics.to_excel(writer, sheet_name='Combined Statistics', index=False)
    
    # Auto-adjust column width for the entire sheet
    worksheet = writer.sheets['Combined Statistics']
    for i, col in enumerate(combined_statistics.columns):
        max_len = max(combined_statistics[col].astype(str).str.len().max(), len(col))
        worksheet.set_column(i, i, max_len)  # Set column width to the max length of the data or column name
    
    # Freeze the first column
    worksheet.freeze_panes(1, 1)  # 'B2' cell is the first unfrozen cell
